In [6]:
## Plotly is not included in your dojo-env
!pip install plotly

In [7]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [8]:
import json
with open('Yelp API/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client_id', 'api-key'])

In [9]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [10]:
# set our API call parameters and filename before the first call
location = 'Atlanta, GA 30033 '
term = 'american'

In [11]:
# Programmatically obtain the city name only from the location string to use in our file name
location.split(',')[0]

'Atlanta'

In [12]:
## Specify folder for saving data
FOLDER = 'Data/'
# This will make the Data folder for us if we don't have one
os.makedirs(FOLDER, exist_ok = True)


# Specifying JSON_FILE filename (can include a folder) to include the location and search term
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

#Verify the name of the JSON_FILE
JSON_FILE

'Data/Atlanta-american.json'

In [13]:
# We can extract the folder name from the file path 
folder = os.path.dirname(JSON_FILE)
folder

'Data'

In [14]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists == False:    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON folder name is not empty: (We have included a folder in our filepath)
    if len(folder)>0:
        # create the folder (In our case, the folder is Data)
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Crating and saving empty list to file.")

    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
          json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Atlanta-american.json not found. Crating and saving empty list to file.


In [15]:
# use our yelp_api variable's search_query method to perform our API call
# for help with any function, go inside () and click shift + tab
# we MUST include location (or lat/long) but we can include other arguments as well

results = yelp_api.search_query(location = location, term = term)

In [16]:
# Check data type of the results
type(results)

dict

In [17]:
# Check the length of the results
len(results)

3

In [18]:
# List the keys of the dictionary
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [19]:
# Explore each key
results['businesses']

[{'id': '-y7EeIOjoFnNjdnBXc23hg',
  'alias': 'the-kitchen-decatur',
  'name': 'The Kitchen',
  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/lfzSmE_pch8lOFv-I3Lhpg/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/the-kitchen-decatur?adjust_creative=7KT0rwzQNkV48V7vYExtqQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=7KT0rwzQNkV48V7vYExtqQ',
  'review_count': 93,
  'categories': [{'alias': 'tradamerican', 'title': 'American'},
   {'alias': 'soulfood', 'title': 'Soul Food'},
   {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}],
  'rating': 4.3,
  'coordinates': {'latitude': 33.77842, 'longitude': -84.27765},
  'transactions': ['delivery', 'pickup'],
  'price': '$$',
  'location': {'address1': '2752 E Ponce De Leon Ave',
   'address2': 'Ste A1',
   'address3': '',
   'city': 'Decatur',
   'zip_code': '30030',
   'country': 'US',
   'state': 'GA',
   'display_address': ['2752 E Ponce De Leon Ave',
    'Ste A1',
    'Decatur, GA 30

In [20]:
# Explore each key
results['total']

802

In [21]:
# Explore each key
results['region']

{'center': {'longitude': -84.287109375, 'latitude': 33.813088376606714}}

In [22]:
# What is the datatype for the businesses?
type(results['businesses'])

list

In [23]:
# Display first result from the list of dictionaries
results['businesses'][0]

{'id': '-y7EeIOjoFnNjdnBXc23hg',
 'alias': 'the-kitchen-decatur',
 'name': 'The Kitchen',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/lfzSmE_pch8lOFv-I3Lhpg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/the-kitchen-decatur?adjust_creative=7KT0rwzQNkV48V7vYExtqQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=7KT0rwzQNkV48V7vYExtqQ',
 'review_count': 93,
 'categories': [{'alias': 'tradamerican', 'title': 'American'},
  {'alias': 'soulfood', 'title': 'Soul Food'},
  {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}],
 'rating': 4.3,
 'coordinates': {'latitude': 33.77842, 'longitude': -84.27765},
 'transactions': ['delivery', 'pickup'],
 'price': '$$',
 'location': {'address1': '2752 E Ponce De Leon Ave',
  'address2': 'Ste A1',
  'address3': '',
  'city': 'Decatur',
  'zip_code': '30030',
  'country': 'US',
  'state': 'GA',
  'display_address': ['2752 E Ponce De Leon Ave',
   'Ste A1',
   'Decatur, GA 30030']},
 'phone': '+1404

In [24]:
## We can convert the a list of dictionaries  to a datafram
df = pd.DataFrame(results['businesses'])
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,-y7EeIOjoFnNjdnBXc23hg,the-kitchen-decatur,The Kitchen,https://s3-media1.fl.yelpcdn.com/bphoto/lfzSmE...,False,https://www.yelp.com/biz/the-kitchen-decatur?a...,93,"[{'alias': 'tradamerican', 'title': 'American'...",4.3,"{'latitude': 33.77842, 'longitude': -84.27765}","[delivery, pickup]",$$,"{'address1': '2752 E Ponce De Leon Ave', 'addr...",+14043433806,(404) 343-3806,3955.605339
1,U4K4dCTtGeNNPUtDaC3HHg,wheelhouse-craft-pub-and-kitchen-decatur,Wheelhouse Craft Pub And Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/GF2sVV...,False,https://www.yelp.com/biz/wheelhouse-craft-pub-...,187,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.6,"{'latitude': 33.792788, 'longitude': -84.290715}",[pickup],$$,"{'address1': '1479 Scott Blvd', 'address2': ''...",+14702404945,(470) 240-4945,2274.825994
2,dfL1KYHtcs6YaFVx-nZTdQ,whiskey-bird-atlanta,Whiskey Bird,https://s3-media4.fl.yelpcdn.com/bphoto/QAGt0x...,False,https://www.yelp.com/biz/whiskey-bird-atlanta?...,642,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,"{'latitude': 33.792656, 'longitude': -84.351852}",[delivery],$$,"{'address1': '1409 N Highland Ave NE', 'addres...",+14045000575,(404) 500-0575,6399.007374
3,yyakxBb3bLHcnIxHUKJNgw,the-general-muir-atlanta,The General Muir,https://s3-media1.fl.yelpcdn.com/bphoto/eFOWuz...,False,https://www.yelp.com/biz/the-general-muir-atla...,1241,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.0,"{'latitude': 33.8009687, 'longitude': -84.3274...",[delivery],$$,"{'address1': '1540 Avenue Pl', 'address2': 'St...",+16789279131,(678) 927-9131,3966.477078
4,bJJK4Hgzdedu98FgtKi56Q,brew-and-bird-decatur-2,Brew and Bird,https://s3-media1.fl.yelpcdn.com/bphoto/rYwox5...,False,https://www.yelp.com/biz/brew-and-bird-decatur...,194,"[{'alias': 'beerbar', 'title': 'Beer Bar'}, {'...",4.3,"{'latitude': 33.792235, 'longitude': -84.30513...","[delivery, pickup]",$$,"{'address1': '1355 Clairmont Rd', 'address2': ...",+14042055566,(404) 205-5566,2858.554068


In [25]:
## How many did we get the details for?
len(df)

20

In [26]:
# How many pages are needed?
n_pages = results['total']/len(df)
n_pages

40.1

In [27]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil(n_pages)
n_pages

41

In [28]:
## Read in results in progress file and check the length
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)

In [29]:
# Preview the contents of the json file (hint, we expect this to be empty at this point)
previous_results

[]

In [30]:
# Determine how many results are present already (hint, we expect this to be 0 at this point
len(previous_results)

0

In [31]:
# Make a loop to extract data from each page and save it to the JSON_FILE

for i in tqdm_notebook(range(n_pages)): 
    ## The block of code we want to TRY to run
    try:
        time.sleep(.2)
     
       
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
            
        ## save number of results to use as offset for query
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp_api.search_query(location = location, term = term, offset = n_results)  # Notice we added the offset to our search_query
        

        ## append new results and save to file  (extend is similar to append, but adds a list to a list)
        previous_results.extend(results['businesses'])

        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.

    except Exception as e:
        print(' [!] ERROR', e)

  0%|          | 0/41 [00:00<?, ?it/s]

In [32]:
# Read the json file in as a df
df = pd.read_json(JSON_FILE)

In [33]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,-y7EeIOjoFnNjdnBXc23hg,the-kitchen-decatur,The Kitchen,https://s3-media1.fl.yelpcdn.com/bphoto/lfzSmE...,False,https://www.yelp.com/biz/the-kitchen-decatur?a...,93,"[{'alias': 'tradamerican', 'title': 'American'...",4.3,"{'latitude': 33.77842, 'longitude': -84.27765}","[pickup, delivery]",$$,"{'address1': '2752 E Ponce De Leon Ave', 'addr...",+14043433806,(404) 343-3806,3955.605339
1,U4K4dCTtGeNNPUtDaC3HHg,wheelhouse-craft-pub-and-kitchen-decatur,Wheelhouse Craft Pub And Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/GF2sVV...,False,https://www.yelp.com/biz/wheelhouse-craft-pub-...,187,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",4.6,"{'latitude': 33.792788, 'longitude': -84.290715}",[pickup],$$,"{'address1': '1479 Scott Blvd', 'address2': ''...",+14702404945,(470) 240-4945,2274.825994
2,dfL1KYHtcs6YaFVx-nZTdQ,whiskey-bird-atlanta,Whiskey Bird,https://s3-media4.fl.yelpcdn.com/bphoto/QAGt0x...,False,https://www.yelp.com/biz/whiskey-bird-atlanta?...,642,"[{'alias': 'newamerican', 'title': 'New Americ...",4.4,"{'latitude': 33.792656, 'longitude': -84.351852}",[delivery],$$,"{'address1': '1409 N Highland Ave NE', 'addres...",+14045000575,(404) 500-0575,6399.007374
3,yyakxBb3bLHcnIxHUKJNgw,the-general-muir-atlanta,The General Muir,https://s3-media1.fl.yelpcdn.com/bphoto/eFOWuz...,False,https://www.yelp.com/biz/the-general-muir-atla...,1241,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.0,"{'latitude': 33.8009687, 'longitude': -84.3274...",[delivery],$$,"{'address1': '1540 Avenue Pl', 'address2': 'St...",+16789279131,(678) 927-9131,3966.477078
4,bJJK4Hgzdedu98FgtKi56Q,brew-and-bird-decatur-2,Brew and Bird,https://s3-media1.fl.yelpcdn.com/bphoto/rYwox5...,False,https://www.yelp.com/biz/brew-and-bird-decatur...,194,"[{'alias': 'beerbar', 'title': 'Beer Bar'}, {'...",4.3,"{'latitude': 33.792235, 'longitude': -84.30513...","[pickup, delivery]",$$,"{'address1': '1355 Clairmont Rd', 'address2': ...",+14042055566,(404) 205-5566,2858.554068


In [34]:
# Verify the .info 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 802 entries, 0 to 801
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             802 non-null    object 
 1   alias          802 non-null    object 
 2   name           802 non-null    object 
 3   image_url      802 non-null    object 
 4   is_closed      802 non-null    bool   
 5   url            802 non-null    object 
 6   review_count   802 non-null    int64  
 7   categories     802 non-null    object 
 8   rating         802 non-null    float64
 9   coordinates    802 non-null    object 
 10  transactions   802 non-null    object 
 11  price          664 non-null    object 
 12  location       802 non-null    object 
 13  phone          802 non-null    object 
 14  display_phone  802 non-null    object 
 15  distance       802 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 94.9+ KB


In [35]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Atlanta-american.csv.gz'